## Import libraries

In [20]:
import pandas as pd
# import sys
# adding Folder to the system path
# sys.path.append('../src/utils/')
# from pii_extraction import pii_extraction

## Read in excel file to pandas

In [171]:
rule_df = pd.read_excel(
    "../data/raw/RiskClassification_Data_Endpoints_V2.xlsx", sheet_name="RiskRules")

# drop the first coulmn
rule_df = rule_df.iloc[:, 1:]
# change server_location to Amaricas to Americas
rule_df["server_location"] = rule_df["server_location"].replace(
    "Amaricas", "Americas")
# fill the empty values with "None"
rule_df = rule_df.fillna("None")
# remove duplicates
rule_df = rule_df.drop_duplicates()
rule_df


,authentication,security_test_category,security_test_result,server_location,hosting_isp,PII,FII,Risk_Label
0,No Authentication,Injections,Fail,Anywhere,Anyone,None,None,Imminent
5,Some Authentication,Broken Authentication,Fail,Anywhere,Anyone,Yes,Yes,Imminent
10,No Authentication,No Test Performed/Available,None,Anywhere,Anyone,Yes,Yes,High
15,Some Authentication,Broken Authentication,Fail,Anywhere,Anyone,No,No,High
20,Some Authentication,All Tests Performed/Available,Pass,Russia,Anyone,Yes,Yes,High
25,Some Authentication,All Tests Performed/Available,Pass,China,Anyone,Yes,Yes,High
30,Some Authentication,No Test Performed/Available,None,Anywhere,Anyone,No,No,Medium
35,Some Authentication,Injections,Fail,Anywhere,Anyone,Yes,Yes,Medium
40,Some Authentication,Buffer Overflow,Fail,Anywhere,Anyone,Yes,Yes,Medium
45,Some Authentication,XML External Enteties,Fail,Anywhere,Anyone,Yes,Yes,Medium


In [172]:
# find the unique values in column "authentication"
auth_col = rule_df["authentication"].unique()
print(f'Unique values in column "authentication": {auth_col}')

# find the unique values in column "security_test_category"
sec_test_col = rule_df["security_test_category"].unique()
print(f'Unique values in column "security_test_category": {sec_test_col}')

# find the unique values in column "security_test_result"
sec_test_res_col = rule_df["security_test_result"].unique()
print(f'Unique values in column "security_test_result": {sec_test_res_col}')

# find the unique values in column "server_location"
server_loc_col = rule_df["server_location"].unique()
print(f'Unique values in column "server_location": {server_loc_col}')

# find the unique values in column "hosting_isp"
host_isp_col = rule_df["hosting_isp"].unique()
print(f'Unique values in column "hosting_isp": {host_isp_col}')

# find the unique values in column "PII"
pii_col = rule_df["PII"].unique()
print(f'Unique values in column "PII": {pii_col}')

# find the unique values in column "FII"
fii_col = rule_df["FII"].unique()
print(f'Unique values in column "FII": {fii_col}')

# find the unique values in column "Risk_Label"
risk_label_col = rule_df["Risk_Label"].unique()
print(f'Unique values in column "Risk_Label": {risk_label_col}')


Unique values in column "authentication": ['No Authentication' 'Some Authentication']
Unique values in column "security_test_category": ['Injections' 'Broken Authentication' 'No Test Performed/Available'
 'All Tests Performed/Available' 'Buffer Overflow' 'XML External Enteties'
 'Cross-Site Scripting']
Unique values in column "security_test_result": ['Fail' 'None' 'Pass']
Unique values in column "server_location": ['Anywhere' 'Russia' 'China' 'Others' 'West Europe' 'Americas']
Unique values in column "hosting_isp": ['Anyone']
Unique values in column "PII": ['None' 'Yes' 'No']
Unique values in column "FII": ['None' 'Yes' 'No']
Unique values in column "Risk_Label": ['Imminent' 'High' 'Medium' 'Low']


In [173]:
api_df = pd.read_excel("../data/processed/df_pii.xlsx")    
api_df = api_df[['api_endpoint_id', 'authentication', 'security_test_category',
                 'security_test_result', 'server_location', 'hosting_isp', 'is_pii', 'is_fii']]
# fill nan values with "None"
api_df = api_df.fillna("None")
# rename the column "is_pii" to "PII"
api_df = api_df.rename(columns={"is_pii": "PII"})
# rename the column "is_fii" to "FII"
api_df = api_df.rename(columns={"is_fii": "FII"})
# replace is_pii true to yes and false to no
api_df["PII"] = api_df["PII"].replace(True, "Yes")
api_df["PII"] = api_df["PII"].replace(False, "No")
# replace is_fii true to yes and false to no
api_df["FII"] = api_df["FII"].replace(True, "Yes")
api_df["FII"] = api_df["FII"].replace(False, "No")


In [174]:
api_df["authentication"].unique()

array(['query', 'OAuth2', 'None', 'none', 'header', 'BasicAuth', 'body',
       'OAuth1', 'path'], dtype=object)

In [175]:
# process column authentication from api_df, replace nan and none with "No Authentication"
api_df["authentication"] = api_df["authentication"].replace(
    "None", "No Authentication")
# replace all value that not "No Authentication" with "Some Authentication"
api_df["authentication"].mask(api_df["authentication"] != "No Authentication", "Some Authentication", inplace=True)
api_df.head()

,api_endpoint_id,authentication,security_test_category,security_test_result,server_location,hosting_isp,PII,FII
0,1,Some Authentication,None,None,United States,"DigitalOcean, LLC",Yes,No
1,2,Some Authentication,None,None,Canada,Google LLC,Yes,No
2,2,Some Authentication,None,None,Canada,Google LLC,Yes,No
3,2,Some Authentication,None,None,Canada,Google LLC,Yes,No
4,2,Some Authentication,None,None,Canada,Google LLC,Yes,No


In [176]:
api_df["security_test_category"].unique()

array(['None', 'Broken Authentication', 'XML External Entities',
       'Cross-Site Scripting', 'Insecure Deserialization',
       'SQL Injection'], dtype=object)

In [177]:
# process column security_test_category from api_df, replace nan with "No Test Performed/Available"
api_df["security_test_category"] = api_df["security_test_category"].replace(
    "None", "No Test Performed/Available")
# replace Injections with "SQL Injection"
api_df["security_test_category"] = api_df["security_test_category"].replace(
    "Injections", "SQL Injection")
# replace "Insecure Deserialization" with "No Test Performed/Available"
api_df["security_test_category"] = api_df["security_test_category"].replace(
    "Insecure Deserialization", "No Test Performed/Available")
api_df.head()


,api_endpoint_id,authentication,security_test_category,security_test_result,server_location,hosting_isp,PII,FII
0,1,Some Authentication,No Test Performed/Available,None,United States,"DigitalOcean, LLC",Yes,No
1,2,Some Authentication,No Test Performed/Available,None,Canada,Google LLC,Yes,No
2,2,Some Authentication,No Test Performed/Available,None,Canada,Google LLC,Yes,No
3,2,Some Authentication,No Test Performed/Available,None,Canada,Google LLC,Yes,No
4,2,Some Authentication,No Test Performed/Available,None,Canada,Google LLC,Yes,No


In [178]:
api_df["security_test_result"].unique()


array(['None', 0.0, 1.0], dtype=object)

In [179]:
# process column security_test_result from api_df, replace false with "Passed"
api_df["security_test_result"] = api_df["security_test_result"].replace(
    0., "Pass")
# replace true with "Failed"
api_df["security_test_result"] = api_df["security_test_result"].replace(
    1., "Fail")
api_df.head()


,api_endpoint_id,authentication,security_test_category,security_test_result,server_location,hosting_isp,PII,FII
0,1,Some Authentication,No Test Performed/Available,None,United States,"DigitalOcean, LLC",Yes,No
1,2,Some Authentication,No Test Performed/Available,None,Canada,Google LLC,Yes,No
2,2,Some Authentication,No Test Performed/Available,None,Canada,Google LLC,Yes,No
3,2,Some Authentication,No Test Performed/Available,None,Canada,Google LLC,Yes,No
4,2,Some Authentication,No Test Performed/Available,None,Canada,Google LLC,Yes,No


In [180]:
# ['Anywhere' 'Russia' 'China' 'Others' 'West Europe' 'Amaricas']
api_df["server_location"].unique()


array(['United States', 'Canada', 'None', 'Germany', 'Netherlands',
       'United Kingdom', 'Ireland', 'Australia', 'India', 'Luxembourg',
       'Sweden', 'France', 'Spain', 'Bangladesh', 'Singapore', 'Russia',
       'Czechia'], dtype=object)

In [181]:
# process column server_location from api_df, replace nan with "Anywhere"
api_df["server_location"] = api_df["server_location"].replace(
    ["None"], "Anywhere")
api_df["server_location"] = api_df["server_location"].replace(
    ["United States", "Canada"], "Americas")
api_df["server_location"] = api_df["server_location"].replace(
    ["United Kingdom", "Ireland", "Germany",
     "Spain", "Luxembourg", "Sweden", "France", "Netherlands"], "West Europe")
# replace everything else with "Others"
api_df["server_location"] = api_df["server_location"].replace(
    ["India", "Bangladesh", "Japan", "Australia",  "Czechia", "Lithuania", "Singapore"], "Others")
api_df


,api_endpoint_id,authentication,security_test_category,security_test_result,server_location,hosting_isp,PII,FII
0,1,Some Authentication,No Test Performed/Available,None,Americas,"DigitalOcean, LLC",Yes,No
1,2,Some Authentication,No Test Performed/Available,None,Americas,Google LLC,Yes,No
2,2,Some Authentication,No Test Performed/Available,None,Americas,Google LLC,Yes,No
3,2,Some Authentication,No Test Performed/Available,None,Americas,Google LLC,Yes,No
4,2,Some Authentication,No Test Performed/Available,None,Americas,Google LLC,Yes,No
...,...,...,...,...,...,...,...,...
1881,2648,Some Authentication,No Test Performed/Available,None,Americas,Amazon Technologies Inc.,No,No
1882,2649,Some Authentication,No Test Performed/Available,None,West Europe,BellSouth.net Inc.,Yes,No
1883,2650,Some Authentication,No Test Performed/Available,None,West Europe,"Amazon.com, Inc.",Yes,No
1884,2651,Some Authentication,No Test Performed/Available,None,West Europe,Amazon Technologies Inc.,Yes,No


In [182]:
# process column hosting_isp from api_df, replace value with "Anyone"
api_df["hosting_isp"] = "Anyone"
api_df.head()

,api_endpoint_id,authentication,security_test_category,security_test_result,server_location,hosting_isp,PII,FII
0,1,Some Authentication,No Test Performed/Available,None,Americas,Anyone,Yes,No
1,2,Some Authentication,No Test Performed/Available,None,Americas,Anyone,Yes,No
2,2,Some Authentication,No Test Performed/Available,None,Americas,Anyone,Yes,No
3,2,Some Authentication,No Test Performed/Available,None,Americas,Anyone,Yes,No
4,2,Some Authentication,No Test Performed/Available,None,Americas,Anyone,Yes,No


In [183]:
# slice row 32 of rule_df
# rule_df = rule_df.iloc[[32]]
# rule_df
api_df


,api_endpoint_id,authentication,security_test_category,security_test_result,server_location,hosting_isp,PII,FII
0,1,Some Authentication,No Test Performed/Available,None,Americas,Anyone,Yes,No
1,2,Some Authentication,No Test Performed/Available,None,Americas,Anyone,Yes,No
2,2,Some Authentication,No Test Performed/Available,None,Americas,Anyone,Yes,No
3,2,Some Authentication,No Test Performed/Available,None,Americas,Anyone,Yes,No
4,2,Some Authentication,No Test Performed/Available,None,Americas,Anyone,Yes,No
...,...,...,...,...,...,...,...,...
1881,2648,Some Authentication,No Test Performed/Available,None,Americas,Anyone,No,No
1882,2649,Some Authentication,No Test Performed/Available,None,West Europe,Anyone,Yes,No
1883,2650,Some Authentication,No Test Performed/Available,None,West Europe,Anyone,Yes,No
1884,2651,Some Authentication,No Test Performed/Available,None,West Europe,Anyone,Yes,No


In [184]:
# write the rule to classify the Risk_Label based on rule_df
def classify_risk(row):
    """
    Check if the row is the same with any row in rule_df
    """
    # remove api_endpoint_id from row series
    row = row.drop(labels=["api_endpoint_id"])
    for i in range(len(rule_df)):
        row_copy = rule_df.iloc[i]
        # drop the column "Risk_Label"
        row_copy = row_copy.drop(["Risk_Label"])
        if row_copy['server_location'] == 'Anywhere':
            # replace row_copy server_location with row server_location
            row_copy['server_location'] = row['server_location']
        if row_copy['security_test_category'] == 'All Tests Performed/Available':
            row_copy['security_test_category'] = row['security_test_category']
        # check if the row is the same with any row in rule_df
        if row_copy.equals(row):
            # return the value in column "Risk_Label"
            return rule_df.iloc[i]["Risk_Label"]
    return "Low Risk"


In [185]:
# remove column risk_label
# api_df.drop(columns=["Risk_Label"], inplace=True)
# remove api_endpoint_id
# api_df.drop(columns=["api_endpoint_id"], inplace=True)
# apply the function classify_risk to each row in api_df
api_df["Risk_Label"] = api_df.apply(classify_risk, axis=1)
# test_df = api_df.iloc[[67]]
# test_df.drop(columns=["Risk_Label"], inplace=True)
# test_df["Risk_Label"] = test_df.apply(classify_risk, axis=1)


In [186]:
api_df


,api_endpoint_id,authentication,security_test_category,security_test_result,server_location,hosting_isp,PII,FII,Risk_Label
0,1,Some Authentication,No Test Performed/Available,None,Americas,Anyone,Yes,No,Low Risk
1,2,Some Authentication,No Test Performed/Available,None,Americas,Anyone,Yes,No,Low Risk
2,2,Some Authentication,No Test Performed/Available,None,Americas,Anyone,Yes,No,Low Risk
3,2,Some Authentication,No Test Performed/Available,None,Americas,Anyone,Yes,No,Low Risk
4,2,Some Authentication,No Test Performed/Available,None,Americas,Anyone,Yes,No,Low Risk
...,...,...,...,...,...,...,...,...,...
1881,2648,Some Authentication,No Test Performed/Available,None,Americas,Anyone,No,No,Medium
1882,2649,Some Authentication,No Test Performed/Available,None,West Europe,Anyone,Yes,No,Low Risk
1883,2650,Some Authentication,No Test Performed/Available,None,West Europe,Anyone,Yes,No,Low Risk
1884,2651,Some Authentication,No Test Performed/Available,None,West Europe,Anyone,Yes,No,Low Risk


In [187]:
# convert api_df.value_counts() to dataframe
api_df.drop(columns=["api_endpoint_id"], inplace=True)
value_count = api_df.value_counts()
# save value_count to excel
value_count.to_excel("../data/processed/risk_rule_count.xlsx")


In [188]:
# save the result to excel
# api_df.to_excel("../data/processed/risk_labeled.xlsx", index=False)
# find unique value in column authentication
api_df["authentication"].unique()

array(['Some Authentication', 'No Authentication'], dtype=object)

In [60]:
api_df


,authentication,security_test_category,security_test_result,server_location,hosting_isp,PII,FII,Risk_Label
0,Some Authentication,No Test Performed/Available,None,Americas,Anyone,Yes,No,Low Risk
1,Some Authentication,No Test Performed/Available,None,Americas,Anyone,Yes,No,Low Risk
2,Some Authentication,No Test Performed/Available,None,Americas,Anyone,Yes,No,Low Risk
3,Some Authentication,No Test Performed/Available,None,Americas,Anyone,Yes,No,Low Risk
4,Some Authentication,No Test Performed/Available,None,Americas,Anyone,Yes,No,Low Risk
...,...,...,...,...,...,...,...,...
1881,Some Authentication,No Test Performed/Available,None,Americas,Anyone,No,No,Low Risk
1882,Some Authentication,No Test Performed/Available,None,West Europe,Anyone,Yes,No,Low Risk
1883,Some Authentication,No Test Performed/Available,None,West Europe,Anyone,Yes,No,Low Risk
1884,Some Authentication,No Test Performed/Available,None,West Europe,Anyone,Yes,No,Low Risk
